# Создание системы предлагающей пользователям новый тариф

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Необходимо построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  
В распоряжении имеются данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

## Анализ файла с данными.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix


try:
    data = pd.read_csv('C:\\Users\\ivan.bayazov\\Desktop\\users_behavior.csv')

#     data = pd.read_csv('/Users/home/YandexProject/data/3/users_behavior.csv')

except:
    data = pd.read_csv('/datasets/users_behavior.csv')


pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 500)

data.info()

**Вывод**

В нашем распоряжении имеется набор данных содержащий информацию о **3214** объектах.  
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  
**сalls** — количество звонков,  
**minutes** — суммарная длительность звонков в минутах,  
**messages** — количество sms-сообщений,  
**mb_used** — израсходованный интернет-трафик в Мб,  
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

**Предобработка данных не требуется.**

## Исследование качества моделей

In [16]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В рамках исследования качества проведем анализ следующих моделей:
- Решающее дерево;
- Случайный лес;
- Логистическая регрессия.

В начале разделим наш набор данных на следующие выборки:
- Обучающая (60%);
- Валидационная (20%);
- Тестовая (20%).

In [17]:
train, test = train_test_split(data, test_size=0.2, random_state=12345)
train, validate = train_test_split(train, test_size=0.25, random_state=12345)
print(f'Количество объектов в исходном наборе: {len(data)}')
print(f'Количество объектов в обучающей выборке: {len(train)}')
print(f'Количество объектов в валидационной выборке: {len(validate)}')
print(f'Количество объектов в тестовой выборке: {len(test)}')

Количество объектов в исходном наборе: 3214
Количество объектов в обучающей выборке: 1928
Количество объектов в валидационной выборке: 643
Количество объектов в тестовой выборке: 643


Разделим наши выборки на признаки (features) и цели (target)

In [18]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']

features_validate = validate.drop(['is_ultra'], axis=1)
target_validate = validate['is_ultra']

features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

### Решающее дерево

Посчитаем модели решающего дерева с различной глубиной (от 1 до 10)

In [19]:
best_model_of_tree = None
best_result_of_tree = 0
best_depth = 0
for depth in range(1, 11):
    model_of_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_of_tree.fit(features_train, target_train)
    predictions = model_of_tree.predict(features_validate)
    result = accuracy_score(target_validate, predictions)
    if result > best_result_of_tree:
        best_model_of_tree = model_of_tree
        best_result_of_tree = result
        best_depth = depth
        
print(f'Accuracy лучшей модели на валидационной выборке: {best_result_of_tree}\nГлубина дерева: {best_depth}')

Accuracy лучшей модели на валидационной выборке: 0.7744945567651633
Глубина дерева: 7


### Случайный лес

Посчитаем модели случайного леса с различными гиперпараметрами количество деревьев (от 1 до 100) и глубиной дерева (от 1 до 10)

In [20]:
best_model_of_forest = None
best_result_of_forest = 0
best_est = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model_of_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_of_forest.fit(features_train, target_train)
        result = model_of_forest.score(features_validate, target_validate)
        if result > best_result_of_forest:
            best_model_of_forest = model_of_forest
            best_result_of_forest = result
            best_est = est
            best_depth = depth

print(f'Accuracy лучшей модели на валидационной выборке:' 
      f'{best_result_of_forest}\nКоличество деревьев: {best_est}\nГлубина дерева: {best_depth}')

Accuracy лучшей модели на валидационной выборке:0.7962674961119751
Количество деревьев: 3
Глубина дерева: 9


### Логистическая регрессия

In [21]:
model_of_log_regress = LogisticRegression(random_state=12345, solver='liblinear') 
model_of_log_regress.fit(features_train, target_train)
result_of_log_regress = model_of_log_regress.score(features_validate, target_validate)

print("Accuracy модели логистической регрессии на валидационной выборке:", result_of_log_regress)

Accuracy модели логистической регрессии на валидационной выборке: 0.6967340590979783


**Вывод**

По результатам тестов, были выбраны лучшие гиперпараметры для каждого типа моделей и полученны следующие результаты:  

|  | Решающее дерево | Случайный лес | Логистическая регрессия |  
|:-----|----:|----:|----:|
|Accuracy |0.774 |0.796 |0.696 |
|Количество деревьев |1 |3 |- |
|Глубина дерева |7 |9 |- |

## Проверка качества моделей

Проверим качество модели на тестовой выборке

In [22]:
print(f'Accuracy модели решающего дерева на тестовой выборке: {best_model_of_tree.score(features_test, target_test):.3f}')
print(f'Accuracy модели случайного леса на тестовой выборке: {best_model_of_forest.score(features_test, target_test):.3f}')
print(f'Accuracy модели логистической регрессии на тестовой выборке: ' 
      f'{model_of_log_regress.score(features_test, target_test):.3f}')

Accuracy модели решающего дерева на тестовой выборке: 0.788
Accuracy модели случайного леса на тестовой выборке: 0.793
Accuracy модели логистической регрессии на тестовой выборке: 0.703


По итогам проверки получаем, что лучший результат дает модель "Случайного леса" с гиперпараметрами "Количество деревьев" равное **3** и "Глубина дерева" равное **9**. Доля правильно предсказанных ответов составляет более **79%**.

## Проверка модели на вменяемость

В начале проверим, достаточно ли хорошо мы разделили данные на выборки

In [23]:
print(f'Доля объектов с тарифом "Ультра" в тренировочной выборке: {(target_train.mean()):.1%}')
print(f'Доля объектов с тарифом "Ультра" в валидационной выборке: {(target_validate.mean()):.1%}')
print(f'Доля объектов с тарифом "Ультра" в тестовой выборке: {(target_test.mean()):.1%}')

Доля объектов с тарифом "Ультра" в тренировочной выборке: 30.5%
Доля объектов с тарифом "Ультра" в валидационной выборке: 31.1%
Доля объектов с тарифом "Ультра" в тестовой выборке: 30.5%


Для всех выборок у нас получилось очень близкое соотношение тарифов. По этому критерию, все хорошо. Посчитаем теперь метрики качества для нашей модели.

In [24]:
predictions = best_model_of_forest.predict(features_test)

print(f'Количество объектов с тарифом "Ультра" в тестовой выборке: ' 
      f'{(target_test.sum())}')
print(f'Количество объектов с тарифом "Ультра" предсказанных моделью в тестовой выборке: ' 
      f'{(predictions.sum())}')
print(f'Доля объектов с тарифом "Ультра" предсказанных моделью в тестовой выборке:' 
      f'{(predictions.mean()):.1%}')

Количество объектов с тарифом "Ультра" в тестовой выборке: 196
Количество объектов с тарифом "Ультра" предсказанных моделью в тестовой выборке: 129
Доля объектов с тарифом "Ультра" предсказанных моделью в тестовой выборке:20.1%


In [25]:
print(f'Доля правильных предсказаний модели по тестовой выборке: {(accuracy_score(target_test, predictions)):.3f}')

Доля правильных предсказаний модели по тестовой выборке: 0.793


In [26]:
print(f'Точность предсказаний модели по тестовой выборке: {(precision_score(target_test, predictions)):.3f}')

Точность предсказаний модели по тестовой выборке: 0.744


In [27]:
print(f'Полнота предсказаний модели по тестовой выборке: {(recall_score(target_test, predictions)):.3f}')

Полнота предсказаний модели по тестовой выборке: 0.490


In [28]:
print(f'Матрица ошибок предсказаний модели по тестовой выборке:\n\n {confusion_matrix(target_test, predictions)}')

Матрица ошибок предсказаний модели по тестовой выборке:

 [[414  33]
 [100  96]]


Подведем итоги проверки вменяемости модели:  
В целом модель на тестовой выборке предсказала 79,3% правильных ответов. Что довольно не плохо.   
При этом модель предсказала 129 (20.1%) значений "Ультра", в действительности их 196 (30.5%). Из них точно предсказанных 74,4 %, что составляет чуть меньше половины.    
Модель на тестовой выборке смогла правильно определить лишь 96 из 196 значений "Ультра".  
Истинно-положительный: 414  
Ложно-положительный: 33  
Ложно-отрицательный: 100  
Истинно-отрицательный: 96  
Модель очень хорошо определяет пользователей тарифа "Смарт" (93%) и плохо распознает тариф "Ультра" (49%).

## Общий вывод

### Цель проекта
Задача — необходимо построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

### Исходные данные
В распоряжении имелись данные о поведении клиентов, которые уже перешли на эти тарифы.  
 
Файл **"users_behavior"** содержал **3214** строк информации.
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  
**сalls** — количество звонков,  
**minutes** — суммарная длительность звонков в минутах,  
**messages** — количество sms-сообщений,  
**mb_used** — израсходованный интернет-трафик в Мб,  
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

**Предобработка данных не требовалась.**

### Проведенные исследования 

В рамках исследования качества был проведен анализ следующих типов моделей:  
- Решающее дерево. Производился перебор гиперпараметра глубины дерева от 1 до 10;  
- Случайный лес. Производился перебор гиперпараметров количество деревьев от 1 до 10 и глубины деревьев от 1 до 10;  
- Логистическая регрессия.  

Исходный набор данных был разделен на выборки в следующих пропорциях:  
- Обучающая (60%);  
- Валидационная (20%);  
- Тестовая (20%).  


По результатам тестов, были выбраны лучшие гиперпараметры для каждого типа моделей и полученны следующие результаты:

|  | Решающее дерево | Случайный лес | Логистическая регрессия |  
|:-----|----:|----:|----:|
|Accuracy |0.774 |0.796 |0.696 |
|Количество деревьев |1 |3 |- |
|Глубина дерева |7 |9 |- |

Далее на вход этим моделям подавалась тестовая выборка. Были полученны следующие результаты:   
- Accuracy модели решающего дерева: 0.788  
- Accuracy модели случайного леса: 0.793  
- Accuracy модели логистической регрессии: 0.703  

По итогам проверки получось, что лучший результат дает модель "Случайного леса" с гиперпараметрами "Количество деревьев" равное 3 и "Глубина дерева" равное 9. Доля правильно предсказанных ответов составила более 79%. 

По итогам проверки вменяемости модели и расчета метрик качества было определено следующее:   
Модель на тестовой выборке предсказала 79,3% правильных ответов. Что довольно не плохо.  
При этом модель предсказала 129 (20.1%) значений "Ультра", в действительности их 196 (30.5%). Из них точно предсказанных 74,4 %, что составляет чуть меньше половины.  
Модель на тестовой выборке смогла правильно определить лишь 96 из 196 значений "Ультра".  

Матрица ошибок:  
- Истинно-положительный: 414  
- Ложно-положительный: 33  
- Ложно-отрицательный: 100  
- Истинно-отрицательный: 96  

Модель очень хорошо определяет пользователей тарифа "Смарт" (93%) и плохо распознает тариф "Ультра" (49%).